# Final Project: Fake News Detection

By Felix Daubner - Hochschule der Medien

Module 'Supervised and Unsupervised Learning' - Prof. Dr.-Ing. Johannes Maucher

## Baseline model - Logistic Regression

To-Do:
- Create baseline model using neural network

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow import keras
from keras.layers import Embedding, Dense, LSTM
import pickle

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

NUM_WORDS=3000
MAX_SEQUENCE_LEN = 57

To being able to compare the results of the machine learning model to be trained, a baseline model will be implemented. The baseline acts as a reference and will be implemented without further exploration, discussion and / or optimization.

As the task to solve is a classification task, a logistic regression is trained and evaluated.

### Prepare data for model training

At first, the data which was preprocessed in the previous notebook is imported into notebook. Still, it needs to be adjusted to train a Logistic Regression model.

In [58]:
#data = pd.read_csv("data/processed.csv", sep=";", index_col=0)

In [64]:
data = pd.read_json("data/processed.json", orient="records", lines=True)

In [22]:
data.head()

,statement,channel_Instagram,channel_Other,channel_TV,channel_TikTok,channel_X,channel_ad,channel_article,channel_blog,channel_campaign,...,channel_presentation,channel_press,channel_social media,channel_speech,channel_talk,channel_video,truth,token,statement_stop,token_stop
0,says sen bob casey dpa is trying to change the...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",says sen bob casey dpa trying change outcome e...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,says the election results are suspicious becau...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",says election results suspicious opponent us s...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,a ballot dump around 4 am in milwaukee shows t...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ballot dump around 4 milwaukee shows wisconsin...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,kari lake is threatening social security and m...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",kari lake threatening social security medicare,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,republican senate candidate sam brown wants to...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",republican senate candidate sam brown wants cu...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [25]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, train_size=0.7, random_state=42)

### Train model

After splitting the data into X (features) and y (target), the data was split into training and test sets. Now, the model is initialized and then trained using only the training data. As there is not much information in the data, the expectations of the model in terms of accuracy are estimated between 55 - 60%.

In [86]:
log = LogisticRegression()
log.fit(X_train_mlp, y_train_mlp)

/Users/felix/anaconda3/envs/dsmml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Evaluate model

Some evaluations are done using first the training and then the test set. A classification report should provide some insights into the  performance of the model which will be the reference for the neural network.

In [89]:
results_train = pd.DataFrame(y_train_mlp, columns=["true"])
results_train["predicted"] = log.predict(X_train_mlp)

results_train["correct"] = results_train["true"] == results_train["predicted"]

In [90]:
results_train[["correct"]].value_counts()

correct
True       8180
False      5929
dtype: int64

In [91]:
print(f"Classification Report of training data:")
print(classification_report(results_train["true"], results_train["predicted"]))

Classification Report of training data:
              precision    recall  f1-score   support

           0       0.57      0.66      0.61      7000
           1       0.60      0.50      0.55      7109

    accuracy                           0.58     14109
   macro avg       0.58      0.58      0.58     14109
weighted avg       0.58      0.58      0.58     14109



In [94]:
results_test = pd.DataFrame(y_test_mlp, columns=["true"])
results_test["predicted"] = log.predict(X_test_mlp)

results_test["correct"] = results_test["true"] == results_test["predicted"]

In [33]:
results_test[["correct"]].value_counts()

correct
True       4226
False      1819
dtype: int64

In [34]:
print(f"Classification Report of test data:")
print(classification_report(results_test["true"], results_test["predicted"]))

Classification Report of test data:
              precision    recall  f1-score   support

           0       0.75      0.61      0.67      3051
           1       0.66      0.79      0.72      2994

    accuracy                           0.70      6045
   macro avg       0.71      0.70      0.70      6045
weighted avg       0.71      0.70      0.70      6045



## Baseline model - Neural Network (LSTM)

As a first baseline model, a Logistic Regression model was trained on only the categorical data but not the statements itself. That's why a second baseline model, a multi-layer-perceptron, is inititalized and trained using the tokenized and padded statements.

Before this can be done, the data has to be transformed into a useful data structure.

### Prepare and save data for training

The data is prepared for the training process by converting the tokenized statements into a numpy array. In this conversion process, only "token" and "truth" are considered, the encoded channel issue columns are dropped from this baseline model.

In [66]:
X_mlp = np.array(data["token"].apply(np.array).to_list())
y_mlp = np.array(data["truth"])

In [77]:
len(X_mlp)

20156

Also, the data is splitted into training and test data.

In [80]:
X_train_mlp, X_test_mlp, y_train_mlp, y_test_mlp = train_test_split(X_mlp, y_mlp, train_size=0.7, random_state=42)

In [68]:
#X_train_mlp = X_train_mlp.reshape(-1, 1)
#X_test_mlp = X_test_mlp.reshape(-1, 1)

Now, the structure of the MLP is defined. Until now, all statements are tokenized which means every word is assigned to a number. This array of numbers represents the statement. Currently, the relationship between those numbers is unknown. This is why an Embedding layer is needed which maps each number representing a word to a multidimensional vector.

A pre-trained Embedding is used from [GloVe](https://nlp.stanford.edu/projects/glove/) which is famous library word embeddings.

In [69]:
word2vec = KeyedVectors.load_word2vec_format("wiki-news-300d-1M.vec")

Embedding matrix

In [81]:
with open("tokenizer/tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)

In [82]:
embedding_dim = 300  
word_index = tokenizer.word_index 
num_words = min(len(word_index) + 1, NUM_WORDS)  

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < num_words:
        if word in word2vec.key_to_index:
            embedding_vector = word2vec[word]
            embedding_matrix[i] = embedding_vector

In [83]:
model = keras.Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LEN, trainable=False))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

In [84]:
model.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])

In [85]:
model.fit(X_train_mlp, y_train_mlp, epochs=20, batch_size=128, validation_data=(X_test_mlp, y_test_mlp))

Epoch 1/20
111/111 [==============================] - 7s 57ms/step - loss: 0.6921 - accuracy: 0.5209 - val_loss: 0.6916 - val_accuracy: 0.5277
Epoch 2/20
111/111 [==============================] - 6s 53ms/step - loss: 0.6905 - accuracy: 0.5360 - val_loss: 0.6902 - val_accuracy: 0.5409
Epoch 3/20
111/111 [==============================] - 6s 54ms/step - loss: 0.6891 - accuracy: 0.5454 - val_loss: 0.6887 - val_accuracy: 0.5563
Epoch 4/20
111/111 [==============================] - 6s 53ms/step - loss: 0.6876 - accuracy: 0.5644 - val_loss: 0.6874 - val_accuracy: 0.5563
Epoch 5/20
111/111 [==============================] - 6s 53ms/step - loss: 0.6861 - accuracy: 0.5724 - val_loss: 0.6856 - val_accuracy: 0.5785
Epoch 6/20
111/111 [==============================] - 6s 55ms/step - loss: 0.6844 - accuracy: 0.5858 - val_loss: 0.6839 - val_accuracy: 0.5824
Epoch 7/20
111/111 [==============================] - 6s 54ms/step - loss: 0.6826 - accuracy: 0.5947 - val_loss: 0.6820 - val_accuracy: 0.5909